# Validación de datos VACUNASCOVIDv2

## Carga de los datos

Este jupyter notebook esta generando los datos de un año en especifico (2022 el parquet part.0.parquet de VACUNASCOVIDv2) para coincidir con las tablas compartidas del  power bi (Tablas agregadas) 

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ValidaciónTablasVACUNASCOVIDv2").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 20:29:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
#spark.stop()

In [2]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/vacunascovid/vacunascovid_ver2_parquet_2022/part.0.parquet")

df=df1_parquet

In [3]:
# Contar el número de registros en el DataFrame 
total_registros = df.count()

# Listar las columnas del DataFrame 
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 20402
Columnas del DataFrame unido: ['IDAnonimizado', 'PersonabasicaID', 'NroDosis', 'Biologico', 'FechaAplicacion', 'CodigoDepartamentoAplicacion', 'DepartamentoAplicacion', 'CodigoMunicipioAplicacion', 'MunicipioAplicacion', 'CodigoDepartamentoResidencia', 'DepartamentoResidencia', 'CodigoMunicipioResidencia', 'MunicipioResidencia', 'TipoDocumentoIdentificacion', 'Sexo', 'Edad', 'TipoRegimenAfiliacion', 'CriterioPriorizacion', 'CAC_HTA', 'CAC_HTA_FechaDiagnostico', 'CAC_DM', 'CAC_DM_FechaDiagnostico', 'CAC_ERC', 'CAC_ERC_FechaDiagnostico', 'CAC_VIH', 'CAC_VIH_FechaDiagnostico', 'CAC_Cancer', 'CAC_Cancer_FechaDiagnostico', 'DefuncionSospechosoCOVID', 'FechaDefuncion', 'CodigoDepartamentoOcurrencia', 'DepartamentoOcurrencia', 'CodigoMunicipioOcurrencia', 'MunicipioOcurrencia', 'CodigoDepartamentoResidenciaFallecido', 'DepartamentoResidenciaFallecido', 'CodigoMunicipioResidenciaFallecido', 'MunicipioResidenciaFallecido', 'SitioDefuncion', 'Recib

In [6]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 ()
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 20402


In [7]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- IDAnonimizado: string (nullable = true)
 |-- PersonabasicaID: string (nullable = true)
 |-- NroDosis: string (nullable = true)
 |-- Biologico: string (nullable = true)
 |-- FechaAplicacion: string (nullable = true)
 |-- CodigoDepartamentoAplicacion: string (nullable = true)
 |-- DepartamentoAplicacion: string (nullable = true)
 |-- CodigoMunicipioAplicacion: string (nullable = true)
 |-- MunicipioAplicacion: string (nullable = true)
 |-- CodigoDepartamentoResidencia: string (nullable = true)
 |-- DepartamentoResidencia: string (nullable = true)
 |-- CodigoMunicipioResidencia: string (nullable = true)
 |-- MunicipioResidencia: string (nullable = true)
 |-- TipoDocumentoIdentificacion: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- TipoRegimenAfiliacion: string (nullable = true)
 |-- CriterioPriorizacion: string (nullable = true)
 |-- CAC_HTA: string (nullable = true)
 |-- CAC_HTA_FechaDiagnostico: string (nullable = true)


# Empieza el proceso de generación de los datos agrupados por atenciones

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonabasicaID','FechaAplicacion', 'Sexo', 'Edad', 'TipoRegimenAfiliacion',
    'NroDosis', 'Biologico', 'CAC_HTA', 'CAC_DM', 'CAC_ERC','CAC_VIH', 'CAC_Cancer', 
    'CausaBasicaDefuncion', 'DefuncionSospechosoCOVID'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaBasicaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaBasicaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaBasicaID').agg(
    F.first('FechaAplicacion').alias('FechaAplicacion'),
    F.first('Sexo').alias('Sexo'),
    F.first('Edad').alias('Edad'),
    F.first('TipoRegimenAfiliacion').alias('TipoRegimenAfiliacion'),
    F.first('NroDosis').alias('NroDosis'),
    F.first('Biologico').alias('Biologico'),
    F.first('CAC_HTA').alias('CAC_HTA'),
    F.first('CAC_DM').alias('CAC_DM'),
    F.first('CAC_ERC').alias('CAC_ERC'),
    F.first('CAC_VIH').alias('CAC_VIH'), 
    F.first('CAC_Cancer').alias('CAC_Cancer'), 
    F.first('CausaBasicaDefuncion').alias('CausaBasicaDefuncion'), 
    F.first('DefuncionSospechosoCOVID').alias('DefuncionSospechosoCOVID'), 
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|PersonabasicaID|FechaAplicacion| Sexo|  Edad|TipoRegimenAfiliacion|NroDosis|           Biologico|CAC_HTA|CAC_DM|CAC_ERC|CAC_VIH|CAC_Cancer|CausaBasicaDefuncion|DefuncionSospechosoCOVID|personaid_count|
+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|      100005989|     2022-01-17|11001|BOGOTÁ|                 D.C.|       2| MODERNA SWITZERL...|      F|     S|     NO|     NO|        NO|                null|                      NO|              1|
|      100013692|     2022-06-17|    M|    33|                    S|       0|             JANSSEN|     NO|    NO|     NO|     NO|        NO|                null|                      NO|  

24/09/02 20:40:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|PersonaBasicaID|FechaAplicacion| Sexo|  Edad|TipoRegimenAfiliacion|NroDosis|           Biologico|CAC_HTA|CAC_DM|CAC_ERC|CAC_VIH|CAC_Cancer|CausaBasicaDefuncion|DefuncionSospechosoCOVID|personaid_count|
+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|      100005989|     2022-01-17|11001|BOGOTÁ|                 D.C.|       2| MODERNA SWITZERL...|      F|     S|     NO|     NO|        NO|                null|                      NO|              1|
|      100013692|     2022-06-17|    M|    33|                    S|       0|             JANSSEN|     NO|    NO|     NO|     NO|        NO|                null|                      NO|  

Se evidencia errores en los parquetes iniciales con los registros de "Bogotá, DC", esto se menciono varias veces pero no se corrigio 

In [10]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( )
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 20399


no se puede validar la cantidad de registros, no se compartio el archivo parquet para la generación del power bi

In [18]:
 df_final =grouped_df


In [19]:
# Contar el número de registros - 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 20399


In [20]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|PersonaBasicaID|FechaAplicacion| Sexo|  Edad|TipoRegimenAfiliacion|NroDosis|           Biologico|CAC_HTA|CAC_DM|CAC_ERC|CAC_VIH|CAC_Cancer|CausaBasicaDefuncion|DefuncionSospechosoCOVID|personaid_count|
+---------------+---------------+-----+------+---------------------+--------+--------------------+-------+------+-------+-------+----------+--------------------+------------------------+---------------+
|      100005989|     2022-01-17|11001|BOGOTÁ|                 D.C.|       2| MODERNA SWITZERL...|      F|     S|     NO|     NO|        NO|                null|                      NO|              1|
|      100013692|     2022-06-17|    M|    33|                    S|       0|             JANSSEN|     NO|    NO|     NO|     NO|        NO|                null|                      NO|  

Se notifico que esta muestra tenia errores desde la fase de exploración, no se corrigieron... 

# Empieza el proceso de generación de tablas VacunasCOVIDv2

# Pagina 1 - Power BI

In [21]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

## 1. Agrupado por año, mes, 'Sexo', 'Edad', 'TipoRegimenAfiliacion',  'NroDosis', 'Biologico','CAC_HTA', 'CAC_DM', 'CAC_ERC','CAC_VIH', 'CAC_Cancer',    'CausaBasicaDefuncion', 'DefuncionSospechosoCOVID'

In [22]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
df2.select('FechaAplicacion').show(100, truncate=False)

+---------------+
|FechaAplicacion|
+---------------+
|2022-01-17     |
|2022-06-17     |
|2022-01-12     |
|2022-01-20     |
|2022-01-13     |
|2022-02-25     |
|2022-05-13     |
|2022-06-06     |
|2022-07-14     |
|2022-04-12     |
|2022-02-09     |
|2022-01-07     |
|2022-11-09     |
|2022-01-24     |
|2022-01-31     |
|2022-08-12     |
|2022-12-15     |
|2022-01-19     |
|2022-02-15     |
|2022-02-04     |
|2022-03-05     |
|2022-03-11     |
|2022-02-12     |
|2022-08-17     |
|2022-02-16     |
|2022-01-18     |
|2022-01-04     |
|2022-02-22     |
|2022-01-11     |
|2022-03-01     |
|2022-04-07     |
|2022-01-11     |
|2022-03-28     |
|2022-02-04     |
|2022-11-04     |
|2022-06-07     |
|2022-08-18     |
|2022-03-04     |
|2022-04-08     |
|2022-04-16     |
|2022-05-11     |
|2022-01-18     |
|2022-02-09     |
|2022-03-15     |
|2022-08-01     |
|2022-02-21     |
|2022-03-10     |
|2022-01-09     |
|2022-02-12     |
|2022-07-15     |
|2022-03-08     |
|2022-01-19     |
|2022-05-1

In [26]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
     'PersonabasicaID','FechaAplicacion', 'Sexo', 'Edad', 'TipoRegimenAfiliacion',
    'NroDosis', 'Biologico', 'CAC_HTA', 'CAC_DM', 'CAC_ERC','CAC_VIH', 'CAC_Cancer', 
    'CausaBasicaDefuncion', 'DefuncionSospechosoCOVID', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaAplicacion' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn(
    'FechaAplicacion',
    F.to_date('FechaAplicacion', 'yyyy-MM-dd')  # Ajustar el formato a 'yyyy-MM-dd'
)    
# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaAplicacion').isNotNull()) & (F.col('FechaAplicacion') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaAplicacion'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaAplicacion'))

# Agrupar por año, mes, sexo y tipo de régimen etc..
result_df = df_filtered.groupBy('Año', 'Mes', 'Sexo', 'Edad', 'TipoRegimenAfiliacion', 
                                'NroDosis', 'Biologico', 'CAC_HTA', 'CAC_DM', 'CAC_ERC','CAC_VIH', 
                                'CAC_Cancer','CausaBasicaDefuncion', 'DefuncionSospechosoCOVID').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------------+------+---------------------+--------+--------------------+--------------------+--------------------+--------------------+----------------+----------+--------------------+------------------------+------------------+--------------------+
| Año|Mes|           Sexo|  Edad|TipoRegimenAfiliacion|NroDosis|           Biologico|             CAC_HTA|              CAC_DM|             CAC_ERC|         CAC_VIH|CAC_Cancer|CausaBasicaDefuncion|DefuncionSospechosoCOVID|Número_de_Personas|Número_de_Atenciones|
+----+---+---------------+------+---------------------+--------+--------------------+--------------------+--------------------+--------------------+----------------+----------+--------------------+------------------------+------------------+--------------------+
|2022|  6|              F|    73|                    C|      -2|              PFIZER|                  NO|                  NO|                  NO|              NO|        NO|                null|              

No es posible validar los datos, no se utilizo el parquet para construir el powerbi.
Los datos tambien tienen problemas de calidad, lo mencionado en la fase de exploracón

In [27]:
spark.stop()